In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer
from pyspark.sql.functions import length, size
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
##dataframe = sqlContext.read.format('csv').options(header='true', inferScheme='true').load('C:/Users/jazmi/OneDrive/Documents/DataMate/db/all.csv')
df = spark.table("all_csv")

df.columns

Out[ 3 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text']

In [3]:
df = df.select('username', 'about_me_text', 'Gender').na.drop()
df_length = df.withColumn('length', length(df['about_me_text']))
df_length.show()

+--------------+--------------------+------+------+
 username| about_me_text|Gender|length|
+--------------+--------------------+------+------+
ShortyFire1984|....................| F| 30|
 ROXYCHIC549|'" I am not an up...| F| 1700|
 tequilagirl69|Laid back lovable...| F| 181|
atxredneckgirl|Tx country music ...| F| 284|
 mel12th|I am here to look...| F| 364|
 Andi010274|Can't get into my...| F| 49|
 nodutch|Music business go...| F| 67|
 lauranmr|Of all the dating...| F| 735|
 smbd75|Likes: fun, adven...| F| 73|
 candipie1989|Single black fema...| F| 385|
 tif1985|I'm on here just ...| F| 309|
 De1964|Please local only...| F| 1118|
 Trini1971|I am a Mom to a 1...| F| 589|
 FiaFia1213|I like to think I...| F| 352|
 SweetJojos| Up for anything| F| 15|
 Tawdra83|I’m a artist. I c...| F| 434|
 LadyTexas254|not looking for a...| F| 241|
 Vigilia79|Hi there! I'm Chr...| F| 650|
 ayoung292|Music is life and...| F| 105|
 Foreverfungrl| Ask me Are| F| 10|
+--------------+--------------------+------+------+
only showing top 20 rows

In [4]:
male_female_to_num = StringIndexer(inputCol='Gender', outputCol='label')

tokened_df = Tokenizer(inputCol="about_me_text", outputCol="token_text")

stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')

hashing = HashingTF(inputCol='token_text', outputCol="hashedValues")

idf = IDF(minDocFreq = 10, inputCol="hashedValues", outputCol="idf_features")


In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_features', 'length'], outputCol='features')

In [6]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[male_female_to_num, tokened_df, stop_remove, hashing, idf, clean_up])

In [7]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_length)
cleaned = cleaner.transform(df_length)

In [8]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262145,[262144],...|
 0.0|(262145,[4054,538...|
 0.0|(262145,[4333,844...|
 0.0|(262145,[9639,390...|
 0.0|(262145,[14,9639,...|
 0.0|(262145,[18976,24...|
 0.0|(262145,[9639,572...|
 0.0|(262145,[1998,961...|
 0.0|(262145,[224647,2...|
 0.0|(262145,[8449,134...|
 0.0|(262145,[4200,576...|
 0.0|(262145,[1536,332...|
 0.0|(262145,[2711,963...|
 0.0|(262145,[329,9639...|
 0.0|(262145,[16332,10...|
 0.0|(262145,[2437,963...|
 0.0|(262145,[13396,16...|
 0.0|(262145,[456,4054...|
 0.0|(262145,[2624,158...|
 0.0|(262145,[109810,1...|
+-----+--------------------+
only showing top 20 rows

In [9]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [10]:
test_results = predictor.transform(testing)
test_results.show(5)

+-----------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 username| about_me_text|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features| features| rawPrediction| probability|prediction|
+-----------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 121121p012|Fun fun fun. Just...| M| 46| 1.0|[fun, fun, fun., ...|[fun, fun, fun., ...|(262144,[8443,360...|(262144,[8443,360...|(262145,[8443,360...|[-237.06106696838...|[0.99999875948742...| 0.0|
 1IHIH|Looking to meet s...| M| 207| 1.0|[looking, to, mee...|[looking, meet, s...|(262144,[9639,192...|(262144,[9639,192...|(262145,[9639,192...|[-580.04675148955...|[1.0,3.9257032398...| 0.0|
1KindOfLove|Hi, my name is Sh...| F| 892| 0.0|[hi,, my, name, i...|[hi,, name, sheil...|(262144,[3358,751...|(262144,[3358,751...|(262145,[9639,158...|[-3028.0127871240...|[1.0,6.0578372449...| 0.0|
 1STREAD|I LIKE MEN....5"1...| F| 91| 0.0|[i, like, men.......|[like, men....5"1...|(262144,[24417,91...|(262144,[24417,91...|(262145,[24417,91...|[-194.24965259435...|[0.99999999991279...| 0.0|
 1cyndi|Hello. I’m new to...| F| 609| 0.0|[hello., i’m, new...|[hello., i’m, new...|(262144,[9639,133...|(262144,[9639,133...|(262145,[9639,163...|[-2187.5490360695...|[1.0,2.2283941737...| 0.0|
+-----------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.356641